# How to use the mb_modelbase package on real data

Welcome to a short introductory example of the mb_modelbase package. Usually mb_modelbase is a part of the Moo-Software and is used as the backend but it can also used without the frontend as a standalone software. 

## The Allbus2016 data set

For the introduction we will use The German General Social Survey [ALLBUS](https://www.gesis.org/en/allbus/allbus-home/). It is from the Leibniz Institute for the Social Sciences (GESIS) and contains all different kinds of variables of a single person like sex, age, income, place of residence, political attitude ... and so on. We would like to learn models on a small subset and execute some operations mb_modelbase to get a brief overview over the functions of mb_modelbase and how to handle them.

Let us import the data set and have a first view:

In [1]:
import pandas as pd
dataset = pd.read_csv('allbus2016.csv', index_col=0)
dataset.head()

,age,sex,educ,income,eastwest,lived_abroad,spectrum
0,47,Female,3,1800,East,No,1
1,52,Male,3,2000,East,No,5
2,61,Male,2,2500,West,No,6
3,54,Female,2,860,West,Yes,1
4,49,Male,3,2500,West,No,6


We see a data set with 7 variables, 4 continuous and 3 categorical.

## Fitting a model and using some basic functions
First step is to import the mb_modelbase package:

In [2]:
import mb_modelbase as mbase

Now we can create an object of a model and fit the data to it. Of course it is hard to validate if the model class make sense or not from here without visual output.

In [5]:
#Create an object of the model class Conditional Gaussians (with categorical and continuous variables)
mymod = mbase.MixableCondGaussianModel("Allbus_CondGauss")
#Fitting process
mymod.fit(df=dataset)

** Mean parameters (direct) **
p [[[ 0.14030384  0.01742627]
  [ 0.256479    0.0665773 ]]

 [[ 0.15996425  0.02323503]
  [ 0.256479    0.0795353 ]]]
mu [[[[ 0.18641874 -0.11944102 -0.41891244 -0.0420817 ]
   [-0.19096579  0.58730211 -0.16113027 -0.44565415]]

  [[-0.06007602 -0.07881634 -0.33469712 -0.06532139]
   [-0.36235841  0.61998339 -0.17173786 -0.29609454]]]


 [[[ 0.14422068 -0.13517125 -0.09098053  0.1703368 ]
   [-0.16467197  0.38899915  0.14196827 -0.13037792]]

  [[ 0.00361768 -0.12166744  0.43259665  0.1278247 ]
   [-0.04358086  0.36777083  0.74385323 -0.08632347]]]]
Sigma [[[[[  9.31204868e-01  -3.13685512e-01   7.64626654e-04  -1.04818872e-01]
    [ -3.13685512e-01   7.84466203e-01   1.13287349e-01  -8.58367820e-02]
    [  7.64626654e-04   1.13287349e-01   2.92648317e-01  -4.44579459e-02]
    [ -1.04818872e-01  -8.58367820e-02  -4.44579459e-02   1.32275897e+00]]

   [[  1.16364297e+00  -6.48499706e-01  -5.81860591e-02  -8.75337270e-02]
    [ -6.48499706e-01   1.05421056e

After the model is fitted to the data, we can execute some functions on the model:

In [6]:
mymod.aggregate("maximum")

(['Female',
  'West',
  'No',
  51.371812159185012,
  3.3806159278761272,
  1326.0000632802612,
  4.2847494570397346],
 ['Female',
  'West',
  'No',
  52.42017937219731,
  3.4735426008968608,
  1737.0363228699553,
  4.4461883408071747])

Why do we get two different maxima? One aggregation is executed on the fitted model and the other one on the data. We have to change the mode-parameter of the model to get the predicted maximum.

In [7]:
mymod.mode = 'model' # or 'data', 'both'
argmax = mymod.aggregate("maximum")
argmax

['Female',
 'West',
 'No',
 51.371812159185012,
 3.3806159278761272,
 1326.0000632802612,
 4.2847494570397346]

To get a better understanding what is meant by executing the query against the model or data, we calculate the density of the distribution for a specific point. This can be done e.g. with the argmax:

In [8]:
mymod.density(argmax)

0.012709581532377863

What happens if we change the mode of the model?

In [9]:
mymod.mode = 'data'
mymod.density(argmax)

0

Why do we get 0 as an answer? Well, the density query against the data corresponds to the number of observations we have of the given point, the argmax was not observed once obviously (who specify his age on 54.593....?). So let us ask for the density of a specific point that we know it exists, for example the first point in the table above:

In [11]:
firstrow = ['Female', 'East', 'No', 47, 3, 1800, 1]
mymod.density(firstrow)

1

This means, the datapoint appears only once in the whole data set. We also can ask for the density of the model for this point:

In [12]:
mymod.mode = 'model'
mymod.density(firstrow)

0.0014730501919687051

To keep our fitted model, we can save it and load it another time:

In [13]:
mymod.save(model=mymod, filename="example_models/Allbus_CondGauss.mdl")
loadmod = mbase.Model.load("example_models/Allbus_CondGauss.mdl")
loadmod.names

['sex', 'eastwest', 'lived_abroad', 'age', 'educ', 'income', 'spectrum']

For this purpose we also can create a modelbase. This is like an abstract object that keeps all the models we have learned. Important is that we have name a directory where we store and load the models from:

In [14]:
#Create the modelbase
mymodelbase = mbase.ModelBase("My Modelbase", load_all=False, model_dir="example_models")
#Add our fittedmodel mymod to the modelbase mymodelbase
mymodelbase.add(mymod) 
#Save it
mymodelbase.save_all_models

<bound method ModelBase.save_all_models of <mb_modelbase.server.modelbase.ModelBase object at 0x7f9225891898>>

## Marginalization and Conditionalization (basic)

Right now our model is way too complex to obtain some useful information. Therefore we would like to marginalize out some dimensions and conditionalize some other dimension on a specific value in order to concentrate on information we are actually interested in.

### 1. Marginalization

At first we show how we remove some dimensions to reduce the model to less variables:

In [15]:
#'keep' for the dimensions you want to keep
mymod_marg = mymod.copy().marginalize(keep=['income', 'sex', 'eastwest']) 
#'remove' for the dimensions you want to remove
mymod_marg = mymod.copy().marginalize(remove=['lived_abroad', 'spectrum', 'educ', 'age', ]) 
mymod_marg.names

['sex', 'eastwest', 'income']

So we really have the 3 dimensions left we have asked for. Now, we can use our basic functions:

In [16]:
argmax = mymod_marg.aggregate("maximum")
argmax

['Female', 'West', 1352.9024977961801]

In [17]:
mymod_marg.density(["Male", "West", 1800])

0.10433855223064581

So we have reduced our model to 3 dimensions without any new fitting process!

### 2. Conditionalization

Now we want to marginalize some variables on specific values to compare two those, in this example we want to compare the income between men and women:

In [18]:
#How to conditionalize a variable on a specific value:
#1. Conditionalize the variables 
mymod_cond_mann = mymod.copy().condition([mbase.Condition("sex", "==", "Male")])
mymod_cond_frau = mymod.copy().condition([mbase.Condition("sex", "==", "Female")])
#2. Marginalize out the dimension
mymod_cond_mann.marginalize(remove=['sex'])
mymod_cond_frau.marginalize(remove=['sex'])

#Alternative: Steps 1 and 2 chained in one line
mymod_cond_mann = mymod.copy().condition([mbase.Condition("sex", "==", "Male")]).marginalize(remove=['sex'])
mymod_cond_frau = mymod.copy().condition([mbase.Condition("sex", "==", "Female")]).marginalize(remove=['sex'])

After we conditionalized the variable 'sex' in one model on 'Mann' and in the other model on 'Frau', we marginalize out the rest of the values to filter unnecessary information and get the aggregation after it:

In [19]:
mymod_cond_mann.marginalize(keep=['income'])
mymod_cond_frau.marginalize(keep=['income'])
[mymod_cond_mann.aggregate("maximum"), mymod_cond_frau.aggregate("maximum")]

[[1917.6936449926814], [1290.4953763614485]]

We can see that we have a large difference in the income between men and women.